In [0]:
# Import functions and libraries
from pyspark.sql.window import Window
from pyspark.sql import functions as sf

In [0]:
df = spark. \
        table("workspace.default.nifty_100_combined_data")

In [0]:
display(df)

In [0]:
# Lag the close by one tick behind
window = Window.partitionBy("ticker").orderBy("date")

lag_close_df = df. \
                withColumn("lag_close", sf.lag("close").over(window))

In [0]:
display(lag_close_df)

In [0]:
# Compute the close to close log return
log_return_df = lag_close_df. \
                    filter(sf.col("lag_close").isNotNull() & (sf.col("lag_close") != 0)). \
                    withColumn("log_return", sf.log(sf.col("close") / sf.col("lag_close")))

In [0]:
display(log_return_df)

In [0]:
# Compute the rolling realised volatility over a 1 hour window
rolling_window = Window.partitionBy("ticker").orderBy("date").rowsBetween(-11, 0)

# Temporary placeholder for number of cols in window
with_count = log_return_df. \
                withColumn("window_count", sf.count("log_return").over(rolling_window))

# Now only apply the function on window_count == 12
realised_vol_df = with_count. \
                    withColumn("realised_rolling_vol", 
                                           sf.when(
                                               sf.col("window_count") == 12, 
                                               sf.sqrt(sf.sum(sf.pow(sf.col("log_return"), 2)).over(rolling_window))
                                           )
                                           .otherwise(sf.lit(None))
                                )

# Now delete the window_count column
realised_vol_df = realised_vol_df. \
                    drop("window_count")

In [0]:
display(realised_vol_df)

Databricks visualization. Run in Databricks to view.

In [0]:
# Store df in DBX workspace DBFS
realised_vol_df. \
    write. \
    mode("overwrite"). \
    saveAsTable("workspace.default.nifty_100_realised_vol")

In [0]:
# Store in Azure Blob storage
spark.conf.set(f"fs.azure.account.key.<sa_name>.blob.core.windows.net", "<sa_key>")

output_path = f"wasbs://<container_name>@<sa_name>.blob.core.windows.net/nifty_100_realised_vol"

In [0]:
# Write the data into a Azure storage account blob 

realised_vol_df. \
            write. \
            mode("overwrite"). \
            parquet(output_path)

# Read the data into a DataFrame
df = spark. \
        read. \
        format("parquet"). \
        load(output_path)

In [0]:
display(df)